In [84]:
import pandas as pd
import numpy as np
import json
import glob
import ast
import gzip
import datetime
from textblob import TextBlob
from sklearn.metrics.pairwise import cosine_similarity


In [85]:
df1_steam = pd.read_parquet("df_games_final.parquet")
df2_reviews = pd.read_parquet("df_reviews_final.parquet")
df3_items = pd.read_parquet("df_items_final.parquet")

#### ANALISIS DE SENTIMIENTO BASANDOME EN LA COLUMNA REVIEW

In [86]:
from textblob import TextBlob

def analisis_sentimiento(text):
    try:
        analysis = TextBlob(str(text))
        if analysis.sentiment.polarity > 0.2:
            return '2'  # positivo
        elif -0.2 < analysis.sentiment.polarity < 0.2:
            return '1'  # neutral
        else:
            return '0'  # negativo
    except:
        return '1'  # neutral si hay un error o si la reseña esta ausente

#aplicamos la función al dataframe y creamos la nueva columna
df2_reviews['analisis_sentimiento'] = df2_reviews['review'].apply(analisis_sentimiento)

#aeemplazamos la columna review con la nueva columna de análisis de sentimiento
df2_reviews.drop('review', axis=1, inplace=True)
df2_reviews.rename(columns={'analisis_sentimiento': 'review'}, inplace=True)


In [87]:
df2_reviews.dtypes

user_id                object
funny                  object
posted         datetime64[ns]
last_edited            object
item_id                object
helpful                object
recommend                bool
review                 object
dtype: object

In [88]:
df3_items.dtypes

item_id              object
item_name            object
playtime_forever    float64
playtime_2weeks     float64
user_id              object
items_count           int64
steam_id             object
user_url             object
dtype: object

In [89]:
df1_steam 

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,0.0,761140.0,NaN,Kotoshiro
1,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,0.0,761140.0,NaN,Kotoshiro
2,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,0.0,761140.0,NaN,Kotoshiro
3,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,0.0,761140.0,NaN,Kotoshiro
4,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,0.0,761140.0,NaN,Kotoshiro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71549,Laush Studio,Indie,Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,NaN,Laush Dmitriy Sergeevich
71550,Laush Studio,Racing,Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,NaN,Laush Dmitriy Sergeevich
71551,Laush Studio,Simulation,Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,NaN,Laush Dmitriy Sergeevich
71552,SIXNAILS,Casual,EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,NaN,"xropi,stev3ns"


In [90]:
df1_steam['id'] = df1_steam['id'].astype(float)      #convierto valores a tipo float para que me deje hacer el merge
df1_steam['price'] = pd.to_numeric(df1_steam['price'], errors='coerce') #convierto con esta funcion ya que hay algunois prices que son de tipo str, ej (free to play)
df3_items['item_id'] = df3_items['item_id'].astype(float)


## PROCEDO A HACER LOS MERGE PARA LOS DATAFRAMES Y LA CREACION DE LAS FUNCIONES


#### CREACION PRIMER FUNCION
cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [91]:
#guardo columnas necesarias en una variable
columnas_df3 = ['item_id', 'user_id']
columnas_df1 = ['id', 'price']

user_price = df3_items[columnas_df3].merge(df1_steam[columnas_df1], left_on='item_id', right_on='id', how='left')
user_total_price = user_price.groupby('user_id')['price'].sum().reset_index()


In [92]:
#guardo columnas necesarias en una variable
columnas_df2 = ['user_id', 'recommend']

df_user_recommend = df3_items[['user_id', 'item_id']].merge(df2_reviews[columnas_df2], on='user_id', how='inner')
df_user_stats = df_user_recommend.groupby('user_id').agg(
    recommend_sum=('recommend', 'sum'),
    items_count=('item_id', 'size')).reset_index()

user_total_price.to_parquet('user_total_price.parquet', engine='pyarrow')#guardamos el df df_user_total_price


df_user_stats.to_parquet('user_stats.parquet', engine='pyarrow')#guardamos el df df_user_stats



In [93]:
def userdata(user_id):
    user_price_data = user_total_price[user_total_price['user_id'] == user_id]
    user_stats_data = df_user_stats[df_user_stats['user_id'] == user_id]
    
    if not user_price_data.empty and not user_stats_data.empty:
        gasto = round(user_price_data['price'].iloc[0], 2)
        recomendacion = user_stats_data['recommend_sum'].iloc[0]
        total_items = user_stats_data['items_count'].iloc[0]
        
        return {
            'Usuario:': user_id,
            'Cantidad de dinero gastado:': gasto,
            'Porcentaje de recomendación:':  round((recomendacion / total_items) * 100, 2),
            'Cantidad de ítems:': total_items
        }
    else:
        return 'Usuario no encontrado'

userdata('76561197970982479')

{'Usuario:': '76561197970982479',
 'Cantidad de dinero gastado:': 6497.29,
 'Porcentaje de recomendación:': 100.0,
 'Cantidad de ítems:': 831}

#### CREACION SEGUNDA FUNCION
cantidad de usuarios que realizaron reviews entre las fechas dadas y, el porcentaje de recomendación de los mismos en base a reviews.recommend.

In [94]:
user_reviews = df2_reviews[['user_id', 'posted', 'recommend']]


In [95]:
def countreviews(inicio: str, final: str):
    mask = (user_reviews['posted'] >= inicio) & (user_reviews['posted'] <= final)
    reviews = user_reviews[mask] #creo una mascara para filtrar las reviews en el rango que elijo

    #usuarios unicos que realizaron reviews 
    usuarios_unicos = reviews['user_id'].nunique()

    #porcentaje recomendacion
    porcentaje_recomendacion = round(reviews['recommend'].sum() / reviews['recommend'].count() * 100,2)

    return {
        'Cantidad de usuarios con reviews:': usuarios_unicos,
        'Porcentaje de recomendación:': round(porcentaje_recomendacion, 2)
    }

countreviews('2011-11-11','2014-12-19')


{'Cantidad de usuarios con reviews:': 16180,
 'Porcentaje de recomendación:': 92.38}

In [96]:
user_reviews.to_parquet('countreviews.parquet', engine='pyarrow')
#guardo el archivo final en formato parquet 

#### CREACION TERCER FUNCION
devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

In [97]:
columnas_df1_funcion_3 = ['id', 'genres']
columnas_df3_funcion_3 = ['item_id', 'playtime_forever']

genres_playtime = df1_steam[columnas_df1_funcion_3].merge(df3_items[columnas_df3_funcion_3], 
left_on='id', right_on='item_id', how='inner')

#procedo a hacer los merge y groupby 
#fuera del df original asi no me queda tan grande y no me genera inconvenientes


In [98]:
ranking = genres_playtime.groupby('genres')['playtime_forever'].sum().reset_index().sort_values(by='playtime_forever', ascending=False)


In [99]:
def genre(genero: str):
    posicion = ranking[ranking['genres'] == genero].index[0] + 1  #busco el genero en el df y su posicion
    return f"Eel genre {genero} se encuentra en el puesto {posicion}"

genre('Action')


'Eel genre Action se encuentra en el puesto 1'

In [100]:
ranking.to_parquet('genre.parquet', engine='pyarrow') #paso el archivo final a formato parquet


#### CREACION CUARTA FUNCION
top 5 de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.

In [101]:
#guardo columnas necesarias en una variable
columnas_df1_funcion_4 = ['id', 'genres'] 
columnas_df3_funcion_4 = ['user_id', 'item_id', 'playtime_forever', 'user_url']


In [103]:
#procedo a hacer los merge y groupby 
#fuera del df original asi no me queda tan grande y no me genera inconvenientes

genero_y_usuario = df1_steam[columnas_df1_funcion_4].merge(df3_items[columnas_df3_funcion_4],   
                                                    left_on='id', right_on='item_id', how='inner')

genero_y_usuario_agrupado = genero_y_usuario.groupby(['genres', 'user_id', 'user_url'])['playtime_forever'].sum().reset_index()


In [104]:
def userforgenre(genre: str):
    genero_filtrado = genero_y_usuario_agrupado[genero_y_usuario_agrupado['genres'] == genre]
    top_usuario = genero_filtrado.sort_values(by='playtime_forever', ascending=False).head(5) #muestro 
                                                                                #los primeros 5 valores
    return top_usuario[['user_id', 'user_url', 'playtime_forever']]


In [105]:
userforgenre('Action')

,user_id,user_url,playtime_forever
50657,Sp3ctre,http://steamcommunity.com/id/Sp3ctre,1699307.0
64497,shinomegami,http://steamcommunity.com/id/shinomegami,1580428.0
49457,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T,1456212.0
51237,Terminally-Chill,http://steamcommunity.com/id/Terminally-Chill,1065742.0
44133,DownSyndromeKid,http://steamcommunity.com/id/DownSyndromeKid,1061193.0


In [106]:
genero_y_usuario_agrupado.to_parquet('userforgenres.parquet', engine='pyarrow') #paso el archivo final a formato parquet


#### CREACION QUINTA FUNCION
cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.

#### CREACION SEXTA FUNCION
según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
